## 线性回归

不使用torch的内置网络/loss/优化函数的实现方式。

假设需要拟合的表达式为
$$
				y = x_1 w_1 + x_2 w_2 + x_3 w_3+b
$$


In [1]:
import torch
import numpy as np
import random
from matplotlib import pyplot as plt

### 随机生成数据集

In [2]:
num_inputs=3
num_examples=500
real_w=[-6, 3.2, 10.5]  # w1,w2,w3真实值
real_b=1.7  # b真实值

# 生成随机输入，为10000*3矩阵
features=torch.randn(num_examples,num_inputs,dtype=torch.float32)

# 根据真实值生成标签
labels = real_w[0] * features[:,0] 
for i in range(1,num_inputs):
    labels += real_w[i]*features[:,i]
labels += real_b

# 给标签加上噪声
labels+=torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float32)

### 以batch形式读取数据

In [3]:
def data_iter(batch_size,features,labels):
    num_examples=len(features)
    
    # 生成大小为num_examples的数值列表并打乱，做到顺序随机读取数据
    index=list(range(num_examples)) 
    random.shuffle(index)
    
    for i in range(0, num_examples, batch_size):
        j=torch.LongTensor(index[i:min(i+batch_size,num_examples)]) # 此处LongTensor相当于索引张量
        
        # 使用index_select(dim,index) 在dim维取序号为index的数据
        # yield中断，保证按随即列表全部取完
        yield features.index_select(0,j), labels.index_select(0,j)  

### 初始化模型参数（随机生成）

In [4]:
w=torch.tensor(np.random.normal(0,0.01,(num_inputs,1)),dtype=torch.float32) # 生成3*1矩阵便于乘法
b=torch.zeros(1,dtype=torch.float32)

# 要求梯度追踪
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

tensor([0.], requires_grad=True)

### 定义模型

In [5]:
def net(X,W,b):
    return torch.mm(X , W)+ b

### 定义Loss函数

In [6]:
def loss(y,real_y):
    return (y-real_y.view(y.size()))**2/2

### 定义SGD函数

In [7]:
def sgd(params, lr, batch_size):
    for param in params:
        param.data -= lr * param.grad / batch_size

### 训练模型

In [8]:
num_epochs=7
lr=0.03
batch_size=10

for epoch in range(num_epochs):
    for X, Y in data_iter(batch_size, features, labels):
        l=loss(net(X,w,b),Y).sum() # Y是数据集中的输出，与net结果相比较。使用sum函数求和转为标量，便于backward计算
        l.backward() # 计算loss函数的梯度
        sgd([w,b],lr,batch_size)

        #梯度清零
        w.grad.data.zero_()
        b.grad.data.zero_()
    epoch_l=loss(net(features,w,b),labels)
    print('epoch %d, loss %f'%(epoch+1,epoch_l.mean().item())) #mean求均值，item将tensor转为数

# 输出结果
print('\n', real_w,'\n',w)
print(real_b,'\n',b)

epoch 1, loss 3.775766
epoch 2, loss 0.198161
epoch 3, loss 0.011271
epoch 4, loss 0.000725
epoch 5, loss 0.000090
epoch 6, loss 0.000050
epoch 7, loss 0.000047

 [-6, 3.2, 10.5] 
 tensor([[-6.0007],
        [ 3.2008],
        [10.4993]], requires_grad=True)
1.7 
 tensor([1.7001], requires_grad=True)
